In [1]:
!pip install gradio==4.29.0

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: python-apt 2.4.0-ubuntu3-zorin1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-apt or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!pip install cv2

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 14.6 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.5 MB/s eta 0:00:00
DEPRECATION: python-apt 2.4.0-ubuntu3-zorin1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-apt or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
!pip install serial

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.0 MB/s eta 0:00:00
DEPRECATION: python-apt 2.4.0-ubuntu3-zorin1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-apt or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [7]:
!pip install pyserial

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 4.3 MB/s eta 0:00:00
DEPRECATION: python-apt 2.4.0-ubuntu3-zorin1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-apt or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [44]:
import gradio as gr
import cv2
import numpy as np
from ultralytics import YOLO
import serial
import serial.tools.list_ports
from PIL import Image

# Load the YOLO model
model = YOLO("yolov8n-seg.pt")  # load an official model

# Check serial ports
available_ports = []
ports = serial.tools.list_ports.comports()
for p in ports:
    available_ports.append(p.device)   

def capture_and_inference(image):       
    # Perform prediction
    results = model(image)
    
    # Get the image with predictions
    img = results[0].plot()
    
    # Create a heatmap
    heatmap = np.zeros((img.shape[0], img.shape[1]), dtype=np.float32)
    
    # Overlay the prediction masks onto the heatmap
    if results[0].masks is not None:
        for mask in results[0].masks.data:
            mask = mask.cpu().numpy().squeeze()  # Convert mask to numpy array and remove single-dimensional entries
            mask_resized = cv2.resize(mask, (heatmap.shape[1], heatmap.shape[0]))  # Resize mask to match heatmap size
            heatmap = cv2.add(heatmap, mask_resized.astype(np.float32))
    
    # Normalize the heatmap
    heatmap = cv2.normalize(heatmap, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    # Convert heatmap to RGB
    heatmap_img = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    
    # Combine the original image with the heatmap
    overlay_img = cv2.addWeighted(img, 0.6, heatmap_img, 0.4, 0)
    
    return overlay_img

def serial_communication(message):
    # This function is a placeholder. You need to initialize 'ser' with the correct port
    # ser = serial.Serial(port, baudrate, timeout=1)
    # Send character 'S' to start the program
    # ser.write(bytearray('S','ascii'))
    # Read line   
    # while True:
    #     bs = ser.readline()
    #     print(bs)
    return "Serial communication placeholder"

# Main Gradio App
with gr.Blocks() as app:
    gr.Markdown("# PAL Controller")
    ## Settings
    gr.Markdown("## 0. 환경 설정")
    with gr.Row():
        with gr.Column(scale=1):
            ## Serial Setting
            gr.Markdown("### 0-1. 시리얼 세팅")
            serial_port = gr.Dropdown(
                available_ports,
                label = "Serial Port",
                info = "Select Serial port"
            )
        with gr.Column(scale=2):
            ## Webcam Setting
            gr.Markdown("### 0-2. 웹캠 세팅")
            inp = gr.Image(sources='webcam', streaming=True)
    
    ## Run & Inference
    gr.Markdown("## 1. 인스턴스 세그멘테이션 결과")
    out = gr.Image(label="Segmentation Result")
    btn = gr.Button("Capture and Run")
    btn.click(fn=capture_and_inference, inputs=inp, outputs=out)
    
    ## Job Scheduling
    gr.Markdown("## 2. 작업 관리")
    gr.Markdown("### 2-1. 작업 필터")
    gr.Markdown("### 2-2. 작업 큐")

    job_status = gr.Textbox(label="Job Status")
    
    ## Serial Communication
    gr.Markdown("## 3. 시리얼 통신 및 수행 결과")
    serial_output = gr.Textbox(label="Serial Communication Result")
    
    # Set up event handlers
    capture_button.click(
        fn=capture_and_inference,
        inputs=[webcam],
        outputs=[output_image, output_heatmap]
    ).then(
        fn=serial_communication,
        inputs=[serial_port],
        outputs=[serial_output]
    )
    
app.launch()

Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.
